<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

# Parameters

In [1]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

## Code (IPFS Connection)

In [2]:
!npm install -g npm@latest
!npm install -g --force --unsafe-perms bcrypt ipfs
!pip install ipfshttpclient


changed 14 packages, and audited 255 packages in 3s

11 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
npm WARN using --force Recommended protections disabled.
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visio

In [19]:
IPFS_PEER="18.157.173.110"
!nohup jsipfs daemon --enable-namesys-pubsub &
!sleep 5
#!jsipfs config Addresses.API /ip4/127.0.0.1/tcp/5002
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Methods '["PUT", "GET", "POST"]'
!jsipfs swarm connect /ip4/$IPFS_PEER/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr

nohup: appending output to 'nohup.out'
connect /ip4/18.157.173.110/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr success


In [14]:
import ipfshttpclient #{IPFS_PEER}
client = ipfshttpclient.connect(f"/ip4/127.0.0.1/tcp/5002/http") 
ipfs_pinner = ipfshttpclient.connect(f"/ip4/{IPFS_PEER}/tcp/5002/http") 

ipfs_root = "/content/ipfs_root"

!rm -r $ipfs_root
!mkdir -p $ipfs_root

def add_and_publish(pin=False, root="/content/ipfs_root"):
  add_res = client.add(root, recursive=True)
  print("add_res",add_res[-1])
  ipfs_hash = add_res[-1]["Hash"]
  ipns_res = client.name.publish("/ipfs/"+ipfs_hash)
  ipns_link = f"/ipns/{ipns_res['Name']}"
  if pin:
    print("pinres",ipfs_pinner.pin.add(ipfs_hash))
  return ipns_link

ipns_link = add_and_publish(pin=True)

print("IPNS_LINK",ipns_link)

add_res <ipfshttpclient.client.base.ResponseBase: {'Name': 'ipfs_root', 'Hash': 'QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn', 'Size': 4, 'Mode': '0755'}>
pinres <ipfshttpclient.client.base.ResponseBase: {'Pins': ['QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn']}>
IPNS_LINK /ipns/QmVGxPaKaMQGNkSZzjepjbK43UARa3NpCaYwZubSzPXvT8


In [15]:
from IPython.display import IFrame

IFrame(src=f'http://localhost:3000/colabasaservice/incolab.html?node={ipns_link}', width=100, height=100)

## CODE (DEEP-DAZE)

In [16]:
!pip install deep-daze --upgrade
!pip install stomp.py imageio-ffmpeg asyncio websockets varname --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)
Requirement already up-to-date: imageio-ffmpeg in /usr/local/lib/python3.7/dist-packages (0.4.3)
Requirement already up-to-date: asyncio in /usr/local/lib/python3.7/dist-packages (3.4.3)
Requirement already up-to-date: websockets in /usr/local/lib/python3.7/dist-packages (8.1)
Requirement already up-to-date: varname in /usr/local/lib/python3.7/dist-packages (0.6.3)


In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

TEXT="A black and white technical illustration of merging a cat and a human through biohacking."
DEFAULT_PARAMS="--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=1234"
GITHUB_REPO="voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)
conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)
conn.subscribe(destination=f'/queue/deep-daze', id=1, ack='auto')

def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  log_out = open(f"{ipfs_root}/colabConsole.txt","a")
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    log_out.write(line.decode("utf-8"))
    log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root
          print("Adding:", added,"to ipfs root")
          #pin=True
    add_and_publish(pin)          

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))


class MyListener(stomp.ConnectionListener):
    def __init__(self, conn,run):
        self.conn = conn
        self.run = run

    def on_error(self, frame):
        print('received an error "%s"' % frame.body)

    def on_message(self, frame):
        print('received a message "%s"' % frame.body)
        self.run(frame.body)
        print('processed message')

    def on_disconnected(self):
        print('disconnected')
        #connect_and_subscribe(self.conn)

conn.set_listener('', MyListener(conn,run))

#run(TEXT)
#conn.disconnect()
sleep(1000000)


voodoohop_colabasaservice_blob_master_colabs_deep_daze_ipynb


In [ ]:
conn.disconnect()